In [1]:
%load_ext autoreload
%autoreload 2

# Testing _crossfit Function

In [2]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.linear_model import Lasso
from econml._ortho_learner import _crossfit

class Wrapper:
    def __init__(self, model):
        self._model = model
    def fit(self, X, y, W=None):
        self._model.fit(X, y)
        return self
    def predict(self, X, y, W=None):
        return self._model.predict(X)
np.random.seed(123)
X = np.random.normal(size=(5000, 3))
y = X[:, 0] + np.random.normal(size=(5000,))
folds = list(KFold(2).split(X, y))
model = Lasso(alpha=0.01)
nuisance, model_list, fitted_inds = _crossfit(Wrapper(model),
                                 folds,
                                 X, y, W=y, Z=None)
print(nuisance)
print(model_list)
fitted_inds

(array([-1.1057289 , -1.53756637, -2.4518278 , ...,  1.10628792,
       -1.82966233, -1.78227335]),)
[<__main__.Wrapper object at 0x131e67358>, <__main__.Wrapper object at 0x11b0aa518>]


array([   0,    1,    2, ..., 4997, 4998, 4999])

# Simple DML with the _OrthoLearner

In [3]:
import numpy as np
from sklearn.linear_model import LinearRegression
from econml._ortho_learner import _OrthoLearner
class ModelNuisance:
    def __init__(self, model_t, model_y):
        self._model_t = model_t
        self._model_y = model_y
    def fit(self, Y, T, W=None):
        self._model_t.fit(W, T)
        self._model_y.fit(W, Y)
        return self
    def predict(self, Y, T, W=None):
        return Y - self._model_y.predict(W), T - self._model_t.predict(W)
class ModelFinal:
    def __init__(self):
        return
    def fit(self, Y, T, W=None, nuisances=None):
        Y_res, T_res = nuisances
        self.model = LinearRegression(fit_intercept=False).fit(T_res.reshape(-1, 1), Y_res)
        return self
    def predict(self, X=None):
        return self.model.coef_[0]
    def score(self, Y, T, W=None, nuisances=None):
        Y_res, T_res = nuisances
        return np.mean((Y_res - self.model.predict(T_res.reshape(-1, 1)))**2)
np.random.seed(123)
X = np.random.normal(size=(100, 3))
y = X[:, 0] + X[:, 1] + np.random.normal(0, 0.1, size=(100,))
est = _OrthoLearner(ModelNuisance(LinearRegression(), LinearRegression()),
                    ModelFinal(),
                    n_splits=2, discrete_treatment=False, random_state=None)
est.fit(y, X[:, 0], W=X[:, 1:])

In [4]:
est.const_marginal_effect()

1.0236499258047582

In [5]:
est.effect(T0=0, T1=1)

array([1.02364993])

In [6]:
est.score(y, X[:, 0], W=X[:, 1:])

0.00727995424098179

In [7]:
est.model_final.model

LinearRegression(copy_X=True, fit_intercept=False, n_jobs=None,
         normalize=False)

In [8]:
est.model_final.model.coef_

array([1.02364993])

In [9]:
est.score_

0.007568302109999707

# Simple DML with Discrete Treatments with the _OrthoLearner

In [10]:
class ModelNuisance:
    def __init__(self, model_t, model_y):
        self._model_t = model_t
        self._model_y = model_y

    def fit(self, Y, T, W=None):
        self._model_t.fit(W, np.matmul(T, np.arange(1, T.shape[1]+1)))
        self._model_y.fit(W, Y)
        return self

    def predict(self, Y, T, W=None):
        return Y - self._model_y.predict(W), T - self._model_t.predict_proba(W)[:, 1:]

class ModelFinal:

    def __init__(self):
        return

    def fit(self, Y, T, W=None, nuisances=None):
        Y_res, T_res = nuisances
        self.model = LinearRegression(fit_intercept=False).fit(T_res.reshape(-1, 1), Y_res)
        return self

    def predict(self):
        # theta needs to be of dimension (1, d_t) if T is (n, d_t)
        return np.array([[self.model.coef_[0]]])

    def score(self, Y, T, W=None, nuisances=None):
        Y_res, T_res = nuisances
        return np.mean((Y_res - self.model.predict(T_res.reshape(-1, 1)))**2)

np.random.seed(123)
X = np.random.normal(size=(100, 3))
import scipy.special
from sklearn.linear_model import LogisticRegression
T = np.random.binomial(1, scipy.special.expit(X[:, 0]))
sigma = 0.01
y = T + X[:, 0] + np.random.normal(0, sigma, size=(100,))
est = _OrthoLearner(ModelNuisance(LogisticRegression(solver='lbfgs'), LinearRegression()), ModelFinal(),
                    n_splits=2, discrete_treatment=True, random_state=None)
est.fit(y, T, W=X)

In [11]:
est.const_marginal_effect()

array([[1.00123159]])

In [12]:
est.effect()

array([1.00123159])

In [13]:
est.score(y, T, W=X)

0.002569588332146612

In [14]:
est.model_final.model.coef_[0]

1.0012315874866917

In [15]:
est.model_final.model

LinearRegression(copy_X=True, fit_intercept=False, n_jobs=None,
         normalize=False)

In [16]:
est.model_final.model.coef_

array([1.00123159])

In [17]:
est.score_

0.0031604059708364245

In [18]:
est.models_nuisance[0]._model_y.coef_

array([1.28171346, 0.03749846, 0.10120681])

# Simple DML with the _RLearner

In [19]:
import numpy as np
from sklearn.linear_model import LinearRegression
from econml._rlearner import _RLearner
from sklearn.base import clone
class ModelFirst:
    def __init__(self, model):
        self._model = clone(model, safe=False)
    def fit(self, X, W, Y, sample_weight=None):
        self._model.fit(np.hstack([X, W]), Y)
        return self
    def predict(self, X, W):
        return self._model.predict(np.hstack([X, W]))
class ModelFinal:
    def fit(self, X, T_res, Y_res, sample_weight=None, sample_var=None):
        self.model = LinearRegression(fit_intercept=False).fit(X * T_res.reshape(-1, 1), Y_res)
        return self
    def predict(self, X):
        return self.model.predict(X)
np.random.seed(123)
X = np.random.normal(size=(1000, 3))
y = X[:, 0] + X[:, 1] + np.random.normal(0, 0.01, size=(1000,))
est = _RLearner(ModelFirst(LinearRegression()),
                ModelFirst(LinearRegression()),
                ModelFinal(),
                n_splits=2, discrete_treatment=False, random_state=None)
est.fit(y, X[:, 0], X=np.ones((X.shape[0], 1)), W=X[:, 1:])

In [20]:
est.const_marginal_effect(np.ones((1,1)))

array([0.99963147])

In [21]:
est.effect(np.ones((1,1)), T0=0, T1=10)

array([9.99631472])

In [22]:
est.score(y, X[:, 0], X=np.ones((X.shape[0], 1)), W=X[:, 1:])

9.736380060274913e-05

In [23]:
est.model_final.model

LinearRegression(copy_X=True, fit_intercept=False, n_jobs=None,
         normalize=False)

In [24]:
est.model_final.model.coef_

array([0.99963147])

In [25]:
est.score_

9.826232040878233e-05

In [26]:
[mdl._model for mdl in est.models_y]

[LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
          normalize=False),
 LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
          normalize=False)]

In [27]:
[mdl._model for mdl in est.models_t]

[LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
          normalize=False),
 LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
          normalize=False)]

# Checking All Good with LinearDMLCateEstimator

In [28]:
from econml.dml import LinearDMLCateEstimator

np.random.seed(123)
X = np.random.normal(size=(1000, 3))
y = X[:, 0] + X[:, 1] + np.random.normal(0, 0.01, size=(1000,))
est = LinearDMLCateEstimator(model_y=LinearRegression(),
                             model_t=LinearRegression())
est.fit(y, X[:, 0], W=X[:, 1:], inference='statsmodels')

In [29]:
est.effect()

array([1.00089549])

In [30]:
est.effect_interval()

(array([0.99404817]), array([1.0077428]))

In [31]:
est.models_y

[LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
          normalize=False),
 LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
          normalize=False)]

In [32]:
est.models_t

[LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
          normalize=False),
 LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
          normalize=False)]

In [33]:
est.coef

array([1.00089549])

In [35]:
est.coef_interval()

(array([0.99404817]), array([1.0077428]))

In [36]:
est.model_final

In [37]:
est.model_final._param_stderr

array([0.00416287])

# DR Learner Based on _OrthoLearner

In [38]:
import numpy as np
from sklearn.linear_model import LinearRegression
from econml._ortho_learner import _OrthoLearner
class ModelNuisance:
    def __init__(self, model_t, model_y):
        self._model_t = model_t
        self._model_y = model_y

    def fit(self, Y, T, X=None, W=None):
        self._model_t.fit(np.hstack([X, W]), np.matmul(T, np.arange(1, T.shape[1]+1)))
        self._model_y.fit(np.hstack([T, X, W]), Y)
        return self

    def predict(self, Y, T, X=None, W=None):
        propensities = self._model_t.predict_proba(np.hstack([X, W]))
        Y_pred = np.zeros((T.shape[0], T.shape[1] + 1))
        T_counter = np.zeros(T.shape)
        Y_pred[:, 0] = self._model_y.predict(np.hstack([T_counter, X, W]))
        Y_pred[:, 0] += (Y - Y_pred[:, 0]) * np.all(T==0, axis=1) / propensities[:, 0]
        for t in np.arange(T.shape[1]):
            T_counter = np.zeros(T.shape)
            T_counter[:, t] = 1
            Y_pred[:, t + 1] = self._model_y.predict(np.hstack([T_counter, X, W]))
            Y_pred[:, t + 1] += (Y - Y_pred[:, t + 1]) * (T[:, t] == 1) / propensities[:, t + 1]
        return Y_pred

class ModelFinal:

    def __init__(self):
        return

    def fit(self, Y, T, X=None, W=None, nuisances=None):
        Y_pred, = nuisances
        self.models_cate = [LinearRegression().fit(X, Y_pred[:, t] - Y_pred[:, 0])
                            for t in np.arange(1, Y_pred.shape[1])]
        return self

    def predict(self, X=None):
        # theta needs to be of dimension (1, d_t) if T is (n, d_t)
        return np.array([mdl.predict(X) for mdl in self.models_cate]).T

np.random.seed(123)
X = np.random.normal(size=(1000, 3))
import scipy.special
from sklearn.linear_model import LogisticRegression
T = np.random.binomial(1, scipy.special.expit(X[:, 0]))
sigma = 0.01
y = (1 + .5*X[:, 0]) * T + X[:, 0] + np.random.normal(0, sigma, size=(1000,))
est = _OrthoLearner(ModelNuisance(LogisticRegression(solver='lbfgs'), LinearRegression()), ModelFinal(),
                    n_splits=2, discrete_treatment=True, random_state=None)
est.fit(y, T, X=X[:, [0]], W=X[:, 1:])

In [39]:
est.const_marginal_effect(X[:10, [0]])

array([[ 0.43111746],
       [ 0.21377249],
       [-0.26176354],
       [ 0.54421168],
       [ 1.76258919],
       [ 0.76761463],
       [ 1.51079693],
       [ 1.76224943],
       [ 0.34418752],
       [ 0.2538734 ]])

In [40]:
[mdl._model_t.coef_ for mdl in est.models_nuisance]

[array([[ 1.07859458, -0.09378512, -0.16819498]]),
 array([[ 0.87520635, -0.07950399,  0.06037872]])]

In [41]:
[mdl._model_y.coef_ for mdl in est.models_nuisance]

[array([ 0.99703455,  1.25799456, -0.00554411,  0.00216083]),
 array([ 1.04547656e+00,  1.21020962e+00,  7.94069500e-04, -9.68240609e-03])]

In [42]:
est.model_final.models_cate[0].coef_

array([0.51667104])

In [43]:
est.model_final.models_cate[0].intercept_

0.9920313527587243

In [119]:
import numpy as np
from sklearn.linear_model import LinearRegression, LassoCV
from econml.drlearner import DRLearner
import scipy.special
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor, RandomForestRegressor

np.random.seed(123)
X = np.random.normal(size=(5000, 3))
T = np.random.binomial(2, scipy.special.expit(X[:, 0]))
sigma = 0.01
y = (1 + .5*X[:, 0]) * T + X[:, 0] + np.random.normal(0, sigma, size=(5000,))
est = DRLearner(model_propensity=GradientBoostingClassifier(),
                model_regression=GradientBoostingRegressor(),
                model_final=LinearRegression(),
                featurizer=None,
                multitask_model_final=True)
est.fit(y, T, X=X, W=None)

In [120]:
est.effect(X[:3], T0=0, T1=1)

array([ 0.48685231,  0.26881666, -0.18528073])

In [121]:
np.hstack([1 + .5*X[:3, [0]], 2*(1 + .5*X[:3, [0]])])

array([[ 0.4571847 ,  0.9143694 ],
       [ 0.24685264,  0.49370529],
       [-0.21333962, -0.42667924]])

In [122]:
est.const_marginal_effect(X[:3])

array([[ 0.48685231,  0.9328053 ],
       [ 0.26881666,  0.54671514],
       [-0.18528073, -0.36790722]])

In [114]:
est.marginal_effect(1, X[:3])

array([[ 0.41814007,  0.94925201],
       [ 0.2166831 ,  0.54275747],
       [-0.09854992,  0.3346042 ]])

In [125]:
[mdl.feature_importances_ for mdl in est.models_regression]

[array([8.52508412e-01, 5.37022723e-06, 2.67965021e-05, 2.43350570e-02,
        1.23124364e-01]),
 array([8.33012311e-01, 9.69352052e-06, 1.93469050e-05, 2.91345496e-02,
        1.37824099e-01])]

In [116]:
[mdl.feature_importances_ for mdl in est.models_propensity]

[array([0.72503316, 0.13991341, 0.13505343]),
 array([0.71079621, 0.1520403 , 0.13716349])]

In [106]:
np.argsort(est.model_cate(T=2).feature_importances_)[-1]

0

In [118]:
np.argsort(est.multitask_model_cate.feature_importances_)

array([1, 2, 0])

In [124]:
est.multitask_model_cate.coef_

array([[ 0.49291139,  0.01168864,  0.00565244],
       [ 0.99018167, -0.01801674,  0.00149827]])

In [52]:
est.model_cate(T=2).coef_

array([ 0.95599289, -0.00493853, -0.00385992])

In [93]:
est.cate_feature_names(['age', 'gender', 'race'])

['age', 'gender', 'race']

In [54]:
est.model_cate(T=2).intercept_

2.005059340332276

In [55]:
est.model_cate(T=1).coef_

array([ 0.48543176, -0.00244637, -0.00256117])

In [56]:
est.model_cate(T=1).intercept_

0.9933224585432593

In [57]:
est.score_

0.2592019899370822

In [58]:
est.score(y, T, X=X, W=None)

0.14028681088385964

In [59]:
import numpy as np
import scipy.special
from econml.drlearner import DRLearner, LinearDRLearner

np.random.seed(123)
X = np.random.normal(size=(1000, 3))
T = np.random.binomial(2, scipy.special.expit(X[:, 0]))
y = (1 + .5*X[:, 0]) * T + X[:, 0] + np.random.normal(size=(1000,))
est = LinearDRLearner()
est.fit(y, T, X=None, W=X, inference='statsmodels')

In [60]:
from econml.utilities import StatsModelsLinearRegression

In [61]:
est.effect()

array([0.86491034])

In [64]:
est.coef(T=1)

array([], dtype=float64)

In [65]:
est.model_cate(T=1).coef_

array([], dtype=float64)

In [66]:
low, up = est.const_marginal_effect_interval()

In [67]:
point = est.const_marginal_effect()

In [68]:
for x, l, p, u in zip(X, low, point, up):
    print("X={}".format(x))
    for it, (ll, pp, uu) in enumerate(zip(l, p, u)):
        print(ll, pp, uu)
        print((1 + .5*x[0])*(it+1))

X=[-1.0856306   0.99734545  0.2829785 ]
0.6772268715192846 0.8649103357239154 1.0525937999285462
0.4571846983497194
1.6932440708855907 1.9226135780335245 2.1519830851814583
0.9143693966994388


In [69]:
est.effect_interval(T0=np.zeros(3), T1=np.array([1, 2, 1]))

(array([0.67722687, 1.69324407, 0.67722687]),
 array([1.0525938 , 2.15198309, 1.0525938 ]))

In [70]:
est.coef_interval(T=2)

(array([], dtype=float64), array([], dtype=float64))

In [72]:
est.intercept_interval(T=2)

(1.6932440708855907, 2.1519830851814583)

In [73]:
est.cate_feature_names(['age', 'gender', 'race'])

['age', 'gender', 'race']

In [74]:
est.effect_interval()

(array([0.67722687]), array([1.0525938]))